In [1]:
from google.colab import drive

In [14]:
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!ls "/content/gdrive/MyDrive/personality_datasets/"

 big_nose_model		      notebooks
 compiled_stacked_coors.csv   personality_images
 compiled_stacked_coors.txt  'Personality Workflow.gdoc'
'Emotion Classifier'	      refined_images
'Folder Directory.gdoc'       shape_predictor_68_face_landmarks.dat
 headshot_jaw2.jpg	      test.csv
 headshot_jaw.jpg	      test_doublechin
 headshot.jpg		      train_images
 images.jpeg		      train_nose
 img_celeba		      train_nose2
 list_attr_celeba.txt	      valid_images
 lookup.csv		      valid_nose
 my_model		      valid_nose2


In [ ]:
import pandas as pd
import os
path = os.getcwd()
print(path)
path = "/content/gdrive/MyDrive/personality_datasets/"
data = pd.read_csv(path+'list_attr_celeba.txt', sep=" ", header=None, names=["Image_Name", "5_o_Clock", "Arched_Eyebrows", "Attractive_Bags_Under_Eyes", "Bald", "Bangs", "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows", "Chubby", "Double_Chin", "Eyeglasses","Goatee", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones", "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pale_Skin", "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair", "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace", "Wearing_Necktie", "Young"])
data = data.drop([0, 1])
data = data.reset_index(drop=True)
print(data)

/content


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


        Image_Name 5_o_Clock  ... Wearing_Necktie Young
0       000001.jpg        -1  ...              -1    -1
1       000002.jpg        -1  ...             NaN     1
2       000003.jpg        -1  ...              -1    -1
3       000004.jpg        -1  ...              -1   NaN
4       000005.jpg        -1  ...              -1    -1
...            ...       ...  ...             ...   ...
202594  202595.jpg        -1  ...              -1    -1
202595  202596.jpg        -1  ...              -1    -1
202596  202597.jpg        -1  ...             NaN     1
202597  202598.jpg        -1  ...               1    -1
202598  202599.jpg        -1  ...              -1    -1

[202599 rows x 40 columns]


In [ ]:

import pandas as pd
import os
import shutil 
import os.path
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

import numpy as np
from sklearn.utils.multiclass import unique_labels
import os

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.applications import VGG19 #For Transfer Learning
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.utils import to_categorical
from pathlib import Path

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        '/content/gdrive/MyDrive/personality_datasets/train_nose2/',
        target_size=(250, 312),
        color_mode = "grayscale",
        batch_size=32,
        class_mode='categorical')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
        '/content/gdrive/MyDrive/personality_datasets/valid_nose2/',
        target_size=(250, 312),
        batch_size=32,
        color_mode="grayscale",
        class_mode='categorical')

Found 9340 images belonging to 2 classes.
Found 3836 images belonging to 2 classes.


In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', input_shape=(250, 312, 1), padding='same'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.MaxPool2D((2,2), strides=(2,2), padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', padding = 'same'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = 'same'))
# cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding = 'same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dense(64, activation='relu'))
cnn.add(tf.keras.layers.Dense(32, activation='relu'))
cnn.add(tf.keras.layers.Dense(8, activation='relu'))
# cnn.add(tf.keras.layers.Dense(32, activation='relu'))
cnn.add(tf.keras.layers.Dense(2, activation='softmax'))

In [ ]:
from keras.optimizers import Adam

opt = Adam(lr = 0.01)
cnn.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
cnn.fit(x=train_generator, validation_data=test_generator, epochs=15 )
cnn.save('/content/gdrive/MyDrive/personality_datasets/big_nose_model2')

Epoch 1/15
292/292 [==============================] - 4124s 14s/step - loss: 1.4762 - accuracy: 0.6920 - val_loss: 0.6124 - val_accuracy: 0.6984
Epoch 2/15
292/292 [==============================] - 103s 353ms/step - loss: 0.6132 - accuracy: 0.6988 - val_loss: 0.6124 - val_accuracy: 0.6984
Epoch 3/15
292/292 [==============================] - 102s 351ms/step - loss: 0.6142 - accuracy: 0.6965 - val_loss: 0.6122 - val_accuracy: 0.6984
Epoch 4/15
292/292 [==============================] - 103s 351ms/step - loss: 0.6153 - accuracy: 0.6953 - val_loss: 0.6150 - val_accuracy: 0.6984
Epoch 5/15
292/292 [==============================] - 103s 352ms/step - loss: 0.6118 - accuracy: 0.6998 - val_loss: 0.6128 - val_accuracy: 0.6984
Epoch 6/15
292/292 [==============================] - 103s 352ms/step - loss: 0.6140 - accuracy: 0.6968 - val_loss: 0.6128 - val_accuracy: 0.6984
Epoch 7/15
292/292 [==============================] - 103s 352ms/step - loss: 0.6149 - accuracy: 0.6959 - val_loss: 0.6122 - 

In [ ]:
# cnn.save('/content/gdrive/MyDrive/personality_datasets/model/my_model')

NameError: ignored

In [ ]:
!ls /content/gdrive/MyDrive/personality_datasets/

'Emotion Classifier'   personality_images   train_images
 img_align_celeba      refined_images	    Untitled2.ipynb
 img_celeba	       test_images	    valid_images


In [ ]:
from google.colab import files
data.to_csv('test.csv')
with open('/content/gdrive/MyDrive/personality_datsets/test.csv', 'w') as f:
  data.to_csv(f)

FileNotFoundError: ignored

In [ ]:
!ls /content/gdrive/MyDrive/personality_datasets/

'Emotion Classifier'   personality_images   train_images
 img_align_celeba      refined_images	    Untitled2.ipynb
 img_celeba	       test_images	    valid_images


In [ ]:
function ClickConnect(){
    console.log("Clicked on connect button"); 
    document.querySelector("Put ID here").click() // Change id here
}
setInterval(ClickConnect,60000)

SyntaxError: ignored

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf


def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (178, 218))  # resize image to match model's expected sizing
    return new_array.reshape(-1, 178, 218, 3)  # return the image with shaping that TF wants.

unique_traits = ['double_chin', 'no_double_chin']
new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/personality_datasets/my_model')

unique_pictures = {'double_chin' : [],
 'no_double_chin' : [],
}

real_pictures = {'double_chin' : [],
 'no_double_chin' : [],
}

correct = 0
total = 0
unique_list = []
for test_file in os.listdir("/content/gdrive/MyDrive/personality_datasets/test_doublechin"):
    # print(test_file)
    prediction = new_model.predict([prepare("/content/gdrive/MyDrive/personality_datasets/test_doublechin/" + test_file)])
    prediction_list = prediction.tolist()
    prediction_list = prediction_list[0]
    max_value = max(prediction_list)
    maximum = -1
    for index in prediction_list:
        if (maximum < index):
            maximum = index
    unique_index = unique_traits[prediction_list.index(maximum)]
    unique_list.append(unique_index)
    if (unique_index == 'double_chin'):
      unique_index = 1
    else:
      unique_index = -1
    
    list1 = data[data['Image_Name'] == test_file].Double_Chin.tolist()
    list1 = list1[0]

    if (unique_index==list1):
      correct += 1
    
    total += 1
    print(total)
    if (total > 200):
      break
    # unique_pictures[unique_index].append("/content/gdrive/MyDrive/personality_datasets/test_doublechin/" + test_file)

print(correct/total)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
0.7412935323383084


In [ ]:

!ls "/content/gdrive/MyDrive/personality_datasets/my_model"

ls: cannot access '/content/gdrive/MyDrive/personality_datasets/my_model': No such file or directory


In [ ]:
prediction = new_model.predict([prepare("/content/gdrive/MyDrive/personality_datasets/test_doublechin/" + '015734.jpg')])
prediction_list = prediction.tolist()
list1 = data[data['Image_Name'] == '015734.jpg'].Double_Chin.tolist()

In [ ]:
print(list1)

['-1']


In [ ]:
unique_list

['no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_chin',
 'no_double_

In [17]:
!ls /content/gdrive/MyDrive/personality_datasets

 all_images		     'Personality Workflow.gdoc'
 big_nose_model		      refined_images
 big_nose_model2	      shape_predictor_68_face_landmarks.dat
 big_nose_model3	      test.csv
 big_nose_model4	      test_doublechin
 compiled_stacked_coors.csv   test_doublechin_nose
 compiled_stacked_coors.txt   test_image
'Emotion Classifier'	      test_image_nose
'Folder Directory.gdoc'       train_full_images
 headshot_jaw2.jpg	      train_full_nose2
 headshot_jaw.jpg	      train_images
 headshot.jpg		      train_nose
 img_celeba		      train_nose2
 list_attr_celeba.txt	      valid_full_images
 lookup.csv		      valid_full_nose2
 my_model		      valid_images
 notebooks		      valid_nose
 personality_images	      valid_nose2


In [12]:
!ls /content/gdrive

MyDrive  Shareddrives
